In [1]:
!pip install transformers datasets accelerate sentencepiece scikit-learn tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.environ["WANDB_DISABLED"] = "true"   # wandb 끄기

import pandas as pd
import numpy as np
import torch

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
DATA_PATH = "/content/drive/MyDrive/project/data/"

train_df = pd.read_csv(DATA_PATH + "train.csv")
val_df   = pd.read_csv(DATA_PATH + "val.csv")
test_df  = pd.read_csv(DATA_PATH + "test.csv")

print(train_df.head())
print(train_df.columns)
print(train_df["label"].value_counts())

                                                text       label
0  12일 경기 화성동탄경찰서에 따르면 이날 오전 화성 동탄신도시의 한 아파트 단지 주...  structural
1  "유치장에 넣어주세요"...거절 당하자 경찰 목에 흉기 휘둘러 공지사항 더 보기 맨 위로    incident
2            [KBS 대전]\n가정폭력 현장서 ‘출생 미신고’ 아동 발견 경찰 수사    incident
3  [앵커]\n\n1393, 보건복지부가 직접 운영하는 자살 예방 상담 전화번호입니다....    incident
4  [KBS 대구]\n경북에서 지난 3년간 가정 폭력 사건이 연평균 9천 건 넘게 발생...      policy
Index(['text', 'label'], dtype='object')
label
policy        338
incident      269
structural     25
Name: count, dtype: int64


In [4]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train_df["label_id"] = le.fit_transform(train_df["label"])
val_df["label_id"]   = le.transform(val_df["label"])
test_df["label_id"]  = le.transform(test_df["label"])

num_labels = len(le.classes_)
print("라벨 종류:", le.classes_)
print("num_labels:", num_labels)

라벨 종류: ['incident' 'policy' 'structural']
num_labels: 3


In [5]:
# HuggingFace Dataset 생성
from datasets import Dataset

train_ds = Dataset.from_pandas(train_df[["text", "label_id"]].reset_index(drop=True))
val_ds   = Dataset.from_pandas(val_df[["text", "label_id"]].reset_index(drop=True))
test_ds  = Dataset.from_pandas(test_df[["text", "label_id"]].reset_index(drop=True))

train_ds = train_ds.rename_column("label_id", "labels")
val_ds   = val_ds.rename_column("label_id", "labels")
test_ds  = test_ds.rename_column("label_id", "labels")

In [6]:
# Tokenizer + Tokenize + Tensor 포맷
from transformers import AutoTokenizer

model_name = "klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=256)

train_ds = train_ds.map(tokenize, batched=True)
val_ds   = val_ds.map(tokenize, batched=True)
test_ds  = test_ds.map(tokenize, batched=True)

train_ds = train_ds.remove_columns(["text"])
val_ds   = val_ds.remove_columns(["text"])
test_ds  = test_ds.remove_columns(["text"])

train_ds.set_format(type="torch", columns=["input_ids", "token_type_ids", "attention_mask", "labels"])
val_ds.set_format(type="torch", columns=["input_ids", "token_type_ids", "attention_mask", "labels"])
test_ds.set_format(type="torch", columns=["input_ids", "token_type_ids", "attention_mask", "labels"])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/632 [00:00<?, ? examples/s]

Map:   0%|          | 0/79 [00:00<?, ? examples/s]

Map:   0%|          | 0/79 [00:00<?, ? examples/s]

In [7]:
# 모델 불러오기
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
!pip install --upgrade transformers

In [9]:
# Trainer 설정
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/project/output/",
    do_eval=True,
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=20,
)

from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_macro": f1_score(labels, preds, average="macro")
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [10]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
20,0.789500
40,0.602000


Step,Training Loss
20,0.789500
40,0.602000
60,0.454700
80,0.432800
100,0.330300
120,0.307100


TrainOutput(global_step=120, training_loss=0.4860633412996928, metrics={'train_runtime': 5772.3941, 'train_samples_per_second': 0.328, 'train_steps_per_second': 0.021, 'total_flos': 249431520006144.0, 'train_loss': 0.4860633412996928, 'epoch': 3.0})

In [11]:
# 성능 평가
trainer.evaluate(test_ds)

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.4694031774997711,
 'eval_accuracy': 0.8481012658227848,
 'eval_f1_macro': 0.578853046594982,
 'eval_runtime': 90.1398,
 'eval_samples_per_second': 0.876,
 'eval_steps_per_second': 0.055,
 'epoch': 3.0}